# Streaming Data Template

### -----


In [3]:
import requests;                                 # import package used to downlaod data from a web URL
import json;
import pandas as pd;
import seaborn as sns;
import matplotlib.pyplot as plt;
import matplotlib;

from pandas.io.json import json_normalize

matplotlib.style.use('ggplot');
%matplotlib inline

C:\Anaconda2\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [167]:
with open('API Keys.txt', 'r') as keyfile:
    apiKeys=keyfile.read().split('\n')
   
url = 'http://backpack.tf/api/IGetPrices/v4/?key=' + apiKeys[0];        # web address of data stream

data_stream = requests.get(url,stream=True);                                    # create an iterable data stream object

rec = data_stream.iter_lines().next().strip() # Necessary before reading data
data = json.loads(rec)            # convert the json record to a python dictionary

In [168]:
def byteify(input):
    if isinstance(input, dict):
        return {byteify(key): byteify(value)
                for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [byteify(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input

data2 = byteify(data)

In [169]:
#Testing box. Useful/helps me remember. Do not remove.

#dfName = json_normalize(data2['response']['items']["Dillinger's Duffel"]['prices']['6']['Tradable']['Craftable']['0'])#['currency', 'difference', 'value', 'last_update'])
# if(dfName[0][0] == 'prices'):
#     print('Hello')
# dfName = json_normalize(data2['response']['items']["Dillinger's Duffel"], 'prices')
# for qualInt in dfName[0]:
#     dfTrade = json_normalize(data2['response']['items'][itemName]['prices'], qualInt)
#dfName

In [170]:
df = json_normalize(data2['response'])

In [171]:
#Just info that the website gives us.

raw_usd_value = df['raw_usd_value']
current_time = df['current_time']
success = df['success']
usd_currency_index = df['usd_currency_index']
usd_currency = df['usd_currency']

In [178]:
#Note: This is only the item data.
#Does not get raw item data (can easily be added if we want)
#Does not get value_high

#THIS WILL TAKE >2 MINUTES TO RUN

#Might be better to start with some array or something with an initial size (16K ish?)
#then convert when it is done

dfItemFinal = pd.DataFrame(columns = ['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex', 'Currency', 'Value', 'Last_Update', 'Difference'])
df = json_normalize(data2['response'], 'items')
for itemName in df[0]:
    dfName = json_normalize(data2['response']['items'], [itemName])
    if(dfName.size != 2):
        raise Exception('More than 2 items for item' + itemName)
    if(dfName[0][0] != 'prices'):
        raise Exception('First row in %s is not prices' % (itemName))
    if(dfName[0][0] != 'prices'):
        raise Exception('Second row in %s is not defindex' % (itemName))
    defIndex = json_normalize(data2['response']['items'][itemName], ['defindex'])[0][0]
    dfPrices = json_normalize(data2['response']['items'][itemName], 'prices')
    for qualInt in dfPrices[0]:
        dfTrade = json_normalize(data2['response']['items'][itemName]['prices'], qualInt)
        
        for tradeable in dfTrade[0]:
            dfCraft = json_normalize(data2['response']['items'][itemName]['prices'][qualInt], tradeable)
        
            for craftable in dfCraft[0]:
                dfPriceIndex = json_normalize(data2['response']['items'][itemName]['prices'][qualInt][tradeable], craftable)
                
                for priceIndex in dfPriceIndex[0]:
                    dfFinalValues = json_normalize(data2['response']['items'][itemName]['prices'][qualInt][tradeable][craftable][priceIndex])
                    
                    currency = dfFinalValues['currency'][0]
                    difference = dfFinalValues['difference'][0]
                    last_update = dfFinalValues['last_update'][0]
                    value = dfFinalValues['value'][0]
                    dfItemFinal.loc[len(dfItemFinal)]=[itemName, defIndex, qualInt, tradeable, craftable, priceIndex, currency, value, last_update, difference]

In [179]:
dfItemFinal.set_index(['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex'])
dfItemFinal = dfNew.drop('Unnamed: 0', axis=1)
dfItemFinal.sort_index(inplace = True)
dfItemFinal.to_csv('itemOut.csv')

In [180]:
dfNew = pd.read_csv('itemOut.csv')
dfNew = dfNew.set_index(['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex'])
dfNew.sort_index(inplace = True)
dfNew.head(8)

Currency  \
Name                        DefIndex QualInt Trade    Craft         PriceIndex            
A Brush with Death          30186    6       Tradable Craftable     0             metal   
A Color Similar to Slate    5052     6       Tradable Craftable     0             metal   
A Deep Commitment to Purple 5031     6       Tradable Craftable     0             metal   
A Distinctive Lack of Hue   5040     6       Tradable Craftable     0              keys   
                                                      Non-Craftable 0             metal   
A Hat to Kill For           30753    6       Tradable Craftable     0             metal   
                                     11      Tradable Craftable     0             metal   
A Head Full of Hot Air      30662    5       Tradable Craftable     91             keys   

                                                                                Value  \
Name                        DefIndex QualInt Trade    Craft         PriceIndex          
A Brush with Death          30186    6       Tradable Craftable     0             5.0   
A Color Similar to Slate    5052     6       Tradable Craftable     0             7.0   
A Deep Commitment to Purple 5031     6       Tradable Craftable     0             8.0   
A Distinctive Lack of Hue   5040     6       Tradable Craftable     0             1.2   
                                                      Non-Craftable 0            11.0   
A Hat to Kill For           30753    6       Tradable Craftable     0             6.0   
                                     11      Tradable Craftable     0            14.0   
A Head Full of Hot Air      30662    5       Tradable Craftable     91           46.0   

                                                                                Last_Update  \
Name                        DefIndex QualInt Trade    Craft         PriceIndex                
A Brush with Death          30186    6       Tradable Craftable     0            1452896758   
A Color Similar to Slate    5052     6       Tradable Craftable     0            1440546430   
A Deep Commitment to Purple 5031     6       Tradable Craftable     0            1452282722   
A Distinctive Lack of Hue   5040     6       Tradable Craftable     0            1454010649   
                                                      Non-Craftable 0            1453070238   
A Hat to Kill For           30753    6       Tradable Craftable     0            1453625734   
                                     11      Tradable Craftable     0            1453940329   
A Head Full of Hot Air      30662    5       Tradable Craftable     91           1449066881   

                                                                                Difference  
Name                        DefIndex QualInt Trade    Craft         PriceIndex              
A Brush with Death          30186    6       Tradable Craftable     0              0.50000  
A Color Similar to Slate    5052     6       Tradable Craftable     0              0.50000  
A Deep Commitment to Purple 5031     6       Tradable Craftable     0              2.83500  
A Distinctive Lack of Hue   5040     6       Tradable Craftable     0             -4.62375  
                                                      Non-Craftable 0             -2.00000  
A Hat to Kill For           30753    6       Tradable Craftable     0             -0.33000  
                                     11      Tradable Craftable     0             14.50000  
A Head Full of Hot Air      30662    5       Tradable Craftable     91           940.50000

In [198]:
#Could be useful if a json ends up with a lot of data in one column

def flattenColumn(input, column):
    column_flat = pd.DataFrame([[i, c_flattened] for i, y in input[column].apply(list).iteritems() for c_flattened in y], columns=['I', column])
    column_flat = column_flat.set_index('I')
    return input.drop(column, 1).merge(column_flat, left_index=True, right_index=True)

In [202]:
# df = json_normalize(data2['response'])
# new_df = flattenColumn(df, 'items.A Brush with Death.defindex')